In [1]:
#Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#To perform Hot Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

#To trasnform data
from datetime import datetime as dt

Using TensorFlow backend.


In [67]:
#Initial RMA dataframe cleaning (check duplicates, drop NaN for key fields, and filter for "RMA Defective Item")
df1 = pd.read_csv('data_proj3.csv')
print("original shape: ", df1.shape)
print("duplicated rows: ", df1[df1.duplicated()].shape[0])
df1 = df1.dropna(subset = ['RMA Create Date'])
print("new shape: ", df1.shape)
df1['RMA Item Transaction Type'].value_counts()

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


original shape:  (198768, 39)
duplicated rows:  0
new shape:  (198767, 39)


RMA Replacement Item          86486
RMA Defective Item            86391
Defective item for FCO         9956
Replacement item for FCO       8700
Actual Defective Received      5459
RMA CE LEAP Onsite             1557
Defective Actual for FCO        216
RMA Conversion Defective          1
RMA Conversion Replacement        1
Name: RMA Item Transaction Type, dtype: int64

In [66]:
df1.isnull().sum()

Serial Number                0
RMA Create Date              0
RMA Item Transaction Type    0
dtype: int64

In [59]:
df2.columns

Index(['IB Material - Key', 'Installed At - Key', 'Serial Number',
       'Shipped Day'],
      dtype='object')

In [ ]:
df1.rename(columns={})

In [68]:
#Filter for RMA Defective Item only
df1 = df1.loc[df1['RMA Item Transaction Type'] == 'RMA Defective Item']
print("defectives only shape: ", df1.shape)

#Reduce Dimensions
# df1 = df1[['Serial Number', 'RMA Create Date', 'RMA Item Transaction Type']]


defectives only shape:  (86391, 39)


Clarify Item No                                0
IBase Number                                   0
RMA Item Count                                 0
Product Family                                 0
Product ID                                     0
Theater                                        0
RMA Request Date                            1516
New Replacement - Delivered On Date        82015
Replacement - Delivered On Date            12297
RMA Create Date                                0
RMA Create Quarter                             0
RMA Create Month                               0
Country                                        0
Service Level                                  0
OTD Status                                     0
OTD Grouping                                   0
OTD Service Level                              0
Header Status                                  0
RMA Type                                       0
RMA Delivery Due Date                       6630
RMA ID              

In [38]:
#Initial Install Base(IB) dataframe cleaning (check duplicates, drop NaN for key fields) 
df2 = pd.read_csv('IB_data_ship_date.csv')
print("original shape: ", df2.shape)
print("duplicated rows: ", df2[df2.duplicated()].shape[0])
df2 = df2.drop_duplicates()
df2 = df2.dropna(subset=['Serial Number'])
print("new shape: ", df2.shape)
df2.isnull().sum()


original shape:  (1337779, 4)
duplicated rows:  12930
new shape:  (1324849, 4)


IB Material - Key     0
Installed At - Key    0
Serial Number         0
Shipped Day           0
dtype: int64

In [36]:
# x = (dt.today()- dt.strptime('7/22/2018', '%m/%d/%Y')).days
# x

2

In [47]:
#Merging Dataframes
#Check data overlap by comparing inner merge size vs RMA df size
df_inner = pd.merge(df2, df1, how="inner", on='Serial Number')
print(len(df1))
print("Inner Join Coverage (%): ", len(df_inner)*100/len(df1), df_inner.shape)

#Merge IB and RMA df with left join
df = pd.merge(df2, df1, how="left", on="Serial Number")
print("Merged Dataframe shape: ", df.shape)
df.head(5)

86391
Inner Join Coverage (%):  0.3958745702677362 (342, 6)
Merged Dataframe shape:  (1324855, 6)


,IB Material - Key,Installed At - Key,Serial Number,Shipped Day,RMA Create Date,RMA Item Transaction Type
0,MXMPC4E3D32XGESFPP,100000596,CADX0717,2016-06-24,NaN,NaN
1,BT8A78CH2-I02,100053925,HC15180092,2016-06-30,NaN,NaN
2,BT8A78CH2-I02,100053925,HC15230332,2016-05-31,NaN,NaN
3,QFX5100-96S-AFO,100135543,VB3715300091,2016-06-30,NaN,NaN
4,EX4300-48T-AFI,100147020,PE3714140135,2016-11-11,NaN,NaN


In [69]:
#Replace non-defective RMA Create Dates with today's date
df['RMA Create Date'].fillna(dt.today(), inplace=True)
df['RMA Item Transaction Type'].fillna(0, inplace=True)
df['RMA Item Transaction Type'].replace("RMA Defective Item", 1, inplace=True)
df.head(5)

,IB Material - Key,Installed At - Key,Serial Number,Shipped Day,RMA Create Date,RMA Item Transaction Type
0,MXMPC4E3D32XGESFPP,100000596,CADX0717,2016-06-24,2018-07-25 10:38:44.679982,0
1,BT8A78CH2-I02,100053925,HC15180092,2016-06-30,2018-07-25 10:38:44.679982,0
2,BT8A78CH2-I02,100053925,HC15230332,2016-05-31,2018-07-25 10:38:44.679982,0
3,QFX5100-96S-AFO,100135543,VB3715300091,2016-06-30,2018-07-25 10:38:44.679982,0
4,EX4300-48T-AFI,100147020,PE3714140135,2016-11-11,2018-07-25 10:38:44.679982,0
5,QFX3500-48S4Q-ACR,100178576,DA664,2016-03-17,2018-07-25 10:38:44.679982,0
6,QFX3500-48S4Q-ACR,100178576,DB774,2016-03-17,2018-07-25 10:38:44.679982,0
7,QFX3500-48S4Q-ACR,100178576,DB778,2016-03-17,2018-07-25 10:38:44.679982,0
8,QFX3500-48S4Q-ACR,100178576,DC032,2016-03-17,2018-07-25 10:38:44.679982,0
9,EX4300-48T-AFI,100178576,PE3715270234,2016-03-17,2018-07-25 10:38:44.679982,0


In [53]:
df.isnull().sum()

IB Material - Key            0
Installed At - Key           0
Serial Number                0
Shipped Day                  0
RMA Create Date              0
RMA Item Transaction Type    0
dtype: int64

0                     1337446
RMA Defective Item        338
Name: RMA Item Transaction Type, dtype: int64

In [105]:
top_df['RMA Item Transaction Type'].replace('RMA Defective Item', "Defective", inplace=True)
top_df['RMA Item Transaction Type'].replace(0, "Not Defective", inplace=True)
top_df.head(5)

,PRODUCT_NUMBER,INSTALLED_AT_ACCOUNT_NAME,INSTALLED_AT_ACCOUNT_ID,RMA Create Date,As Built2 Ship Date,RMA Item Transaction Type
0,SFP-1GE-LX,AT&T,1.00511e+08,2018-07-24 21:10:02.740987,NaN,Not Defective
1,QFX5100-FAN-AFO,YALE UNIVERSITY,1.01305e+08,2018-07-24 21:10:02.740987,NaN,Not Defective
2,JPSU-650W-AC-AFO,UNIVERSITY OF TEXAS - ARLINGTON,1.01358e+08,2018-07-24 21:10:02.740987,NaN,Not Defective
3,SFPP-10GE-LR,CHUNGHWA TELECOM,1.00852e+08,2018-07-24 21:10:02.740987,NaN,Not Defective
4,SFP-1GE-SX,ERICSSON INC.,1.00562e+08,2018-07-24 21:10:02.740987,NaN,Not Defective


In [10]:
#Loop through each row and calculate lifespan from Built Ship Date and RMA Create Date ('Date Defective' - 'Built Date)
lifespan = []
for time in top_df.index:
    try:
        life = dt.strptime(top_df.iloc[time]['RMA Create Date'], '%m/%d/%Y') - dt.strptime(top_df.iloc[time]['As Built2 Ship Date'], '%m/%d/%Y')
        lifespan.append(life.days)
    except TypeError:
        life = 'NaN'
        lifespan.append(life)


In [124]:
#Add Lifespan to dataframe
top_df['Lifespan(days)'] = pd.DataFrame(lifespan)
print(top_df.shape)
top_df.head(5)

(29299, 5)


,Product ID,RMA Ship Site ID,RMA Create Date,As Built2 Ship Date,Lifespan(days)
0,MX960-PWR4100-AC-S,TALKTALK TELECOM,12/6/2017,12/8/2013,1459
1,MX960-PWR4100-DC-S,KILOMETER,3/15/2018,11/25/2013,1571
2,MPC-3D-16XGE-SFPP,AMAZON.COM,4/21/2017,3/27/2011,2217
3,MPC-3D-16XGE-SFPP,AMAZON.COM,12/1/2017,5/12/2013,1664
4,MXMPC4E3D32XGESFPP,AMAZON.COM,1/24/2017,1/3/2017,21


In [141]:
%matplotlib inline
import matplotlib.pyplot as plt

#Plot Product ID vs Lifespan

#Plot Site ID vs Lifespan

In [125]:
# For loop to go through each product... *** will be built out later ***
top_products

['EX3300-48P',
 'MPC7E-MRATE',
 'MPC-3D-16XGE-SFPP',
 'MX960-PWR4100-DC-S',
 'MX2K-MPC6E',
 'EX2200-48P-4G',
 'MX960-PWR4100-AC-S',
 'EX4200-48T',
 'EX2200-24P-4G',
 'MXMPC4E3D32XGESFPP']

In [129]:
# FIlter by ONE PRODUCT... *** later should make for loop for each of top ten products
first_df = top_df.loc[top_df['Product ID']== 'EX3300-48P']

#Reduce dimensions to just Ship ID and Lifespan
first_df = first_df[['RMA Ship Site ID', 'Lifespan(days)']]
first_df.reset_index(inplace=True)
first_df = first_df.drop(['index'], axis=1)
print(first_df.shape)
first_df.head()

(4530, 2)


,RMA Ship Site ID,Lifespan(days)
0,AON FRANCE,371
1,SECUREMATICS,315
2,MRC OF ICT,132
3,TELSTRA--DOD,151
4,UNIVERSITY OF,699


In [135]:
model_df = pd.get_dummies(first_df['RMA Ship Site ID'])
model_df['Lifespan(days)'] = first_df['Lifespan(days)']
model_df.head(5)

,1E LIMITED,21ST MORTGAGE,24/7 CUSTOMER,A O N,ACCENTURE,ACCENTURE FINANCE,ACUITY BRANDS,ACUMENT,ACUMENT GLOBAL,ADACTA HOLDING,...,YUBI,ZANESVILLE CITY,ZDRAVSTVENI DOM,ZEBRA A.G.R.,ZEBRA STOCKING,ZF CHASSIS SYSTEMS,ZF ELECTRONICS,ZF FRIEDRICHSHAFEN,ZILLOW,Lifespan(days)
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,371
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,315
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,132
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,151
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,699


In [137]:
y = model_df['Lifespan(days)']
X = model_df.drop('Lifespan(days)', axis=1)
print(X.shape, y.shape)

(4530, 799) (4530,)


In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [140]:
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.3487214000289939
Testing Score: -8.531649011204819e+25


In [143]:

model2 = Sequential()
number_inputs = 799
number_hidden_nodes = 4
number_classes = 2 
model2.add(Dense(units=number_hidden_nodes,
               activation='relu', input_dim=number_inputs))
model2.add(Dense(units=number_classes,
               activation='softmax'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 3200      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 3,210
Trainable params: 3,210
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#OLD CODE

# #Sort by highest count of Product ID
# products = pd.DataFrame(df['PRODUCT_NUMBER'].value_counts().reset_index(drop=False))

#---
# #Convert Top 10 Products to list and filter DF by only Top 10 Products
# top_products = products['index'][:10].tolist()
# print(top_products)
# top_df = df.loc[df['PRODUCT_NUMBER'].isin(top_products)]
# print(top_df.shape)

#---
# ###### FOR TRAIL WITH REDUCED DATASET

# #Reduce DF dimensions and reset index
# top_df = top_df[['PRODUCT_NUMBER', 'INSTALLED_AT_ACCOUNT_NAME', 'INSTALLED_AT_ACCOUNT_ID', 'RMA Create Date', 'As Built2 Ship Date', 'RMA Item Transaction Type']]
# top_df.reset_index(inplace=True)
# top_df = top_df.drop('index', axis=1)
# #Remove NaN from date fields
# # top_df = top_df.dropna(subset = ['RMA Create Date', 'As Built2 Ship Date'])
# # top_df.reset_index(inplace=True)
# # top_df = top_df.drop(['index'], axis=1)
# print(top_df.shape)
# top_df.head(5)